# New version

In [3]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", 
                                          nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", 
                                          "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP")
    return salyr

In [4]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)


In [5]:
# Create list of urban centres
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3.shp')

# Specify the columns you need to extract and apply conditions on
columns = ["Batch", "CNTR_CODE", "HDENS_CLST"]

# Initialize an empty list to store the values
values_list = []

# Use an arcpy.da.SearchCursor to iterate over the rows in the shapefile
with arcpy.da.SearchCursor(uc_file_path, columns) as cursor:
    for row in cursor:
        # Apply the selection criteria
        if row[0] == 1.0:
            values_list.append(row[2])

In [6]:
# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400

In [7]:
# Define the path and name for the geodatabase
gdb_path = step2_f
gdb_name = "SA_output_batch1_v1209_v2.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

In [8]:
# Create a list of files in the directory that end with GQA.shp
GQA_ls = [
    os.path.join(step2_f, f) for f in os.listdir(step2_f)
    if f.endswith('finalGQA.shp')
]

In [9]:
len(GQA_ls)

100

In [9]:
import pandas as pd
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

In [10]:
# Crosswalk table containing the different codes from input sources
tbl_path_path = r'S:\Common workspace\Noise\2021\3251_QuietAreas\InputData\QA2021\00_Data_List\tbl_agg_FUA.csv'
tbl_agg_FUA = pd.read_csv(tbl_path_path)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x81 in position 839: invalid start byte

In [11]:
codes.query("HDENS_CLST=='GEOSTAT21_029'")

,FREQUENCY,HDENS_CLST,HDENS_NAME,HDENS_2011,agglomerationId_identifier,agglomerationName_nameEng,UA2018
23,1,GEOSTAT21_029,Göteborg,GEOSTAT11_029,SE_a_ag1480,Gothenburg,SE002L1_GOTEBORG_UA2018_v013


In [22]:
arcpy.CheckInExtension("Network")
if arcpy.CheckExtension("Network") == "Available":
    arcpy.CheckOutExtension("Network")
else:
    raise CustomError("The Network Analyst extension is not available.")

In [23]:
extension_status = arcpy.CheckExtension("Network")
print(extension_status)

Available


In [ ]:
arcpy.CheckOutExtension("Network")

In [ ]:
arcpy.CheckExtension("Network")

In [ ]:
available_extensions = arcpy.ListExtensions()
print(available_extensions)

In [12]:
#Environment settings
arcpy.env.overwriteOutput = 'true'
try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3.shp')
#sql_query = "Batch = 1"
sql_query = "HDENS_CLST = 'GEOSTAT21_029'"
with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        if os.path.exists(inGQA) and ctr_cd!="NO":
            print(inGQA)
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))

            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                ##arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='NO_END_POINTS')

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                # save selection in a diferent feature class
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                # Adding FID field to the point feature class if it doesn't already exist
                if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                    arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +str(len(ids_ls_GQA)))

                # GQA without access to network (less than 25m)
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                
                # List to hold centroid points and their corresponding FIDs
                centroid_points = []
                fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                # Create a search cursor to iterate through the polygon shapefile GQA
                with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                    for row in cursor:
                        if row[0] in ids_noaccess:
                            # Get the FID and the geometry of the polygon
                            fid = row[0]
                            polygon = row[1]
                            area = row[2]
                            # Compute the centroid of the polygon
                            centroid = polygon.centroid
                            # Append the FID and centroid to the list
                            centroid_points.append((fid, centroid, area))

                # Start an edit session for the geodatabase
                with arcpy.da.Editor(outGDB) as edit:
                    # Create an insert cursor for the point feature class
                    with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                        for fid, point, area in centroid_points:
                            # Insert the centroid point and the FID
                            cursor.insertRow([fid, point, area])
                print("Centroids and FIDs computed and appended successfully.")

                # Add FID value to all records
                field_to_check = 'FID'
                alternative_field = 'ORIG_FID'
                #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                expression ="!FID! if !FID! is not None else !ORIG_FID!"
                # Execute CalculateField
                arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)

                
                #Loop through to prevent memory errors
                # Batch processing to prevent memory issues
                batch_size = 1500 
                print(f'batch size {batch_size}')

                # Function to create service area layer
                for i in range(0, int(nbrec), batch_size):
                    j = min(i + batch_size, int(nbrec))
                    print ("this is i {} and this is j {}".format(i,j))
                    expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                    if arcpy.Exists("GUA_points_layer"):
                        arcpy.Delete_management("GUA_points_layer")
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
                    print('GUA_pt_lyr')

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                    salyr.saveACopy(fc_SA)

                    arcpy.Delete_management(salyr) 
 

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)
            finally:
                arcpy.CheckInExtension("Network")




Network Analyst extension has been checked out.
GEOSTAT21_029
Göteborg
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\step2_CleaningGQA\GEOSTAT21_029_finalGQA.shp
Extent set to match uc: 4431000 3837000 4448000 3855000 NaN NaN NaN NaN
nb record(s) in GEOSTAT21_029_finalGQA.shp subset selection [code_2018] IN (14100,30000,31000) : 728
Create points from GUA borders used to create SA
GeneratePointsAlongLines Distance 50m from selected GUA in GEOSTAT21_029_finalGQA.shp to GEOSTAT21_029_GUA_pt_50m
nb generated points : 10043
SelectLayerByLocation points in GEOSTAT21_029_GUA_pt_50m WITHIN_A_DISTANCE of 25 meters from nw to GEOSTAT21_029_GUA_pt_50m_nw_25m
Nb of GQA near network 591
Total nb of GQA 728
Centroids and FIDs computed and appended successfully.
nb points in GEOSTAT21_029_GUA_pt_50m_nw_25m : 5267
batch size 1500
this is i 0 and this is j 1500
GUA_pt_lyr
Creating network layer for batch GEOSTAT21_029
Solving network analysis
ArcPy ERRORS:


In [18]:
uc_cursor

In [21]:
uc_file_path

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\UrbanCentres\\HDC2021_RG_InputUpdateB2B3.shp'

In [ ]:
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\UrbanCentres\HDC2021_RG_InputUpdateB2B3.shp

In [ ]:
arcpy.env.addOutputsToMap = False

## All in one cell

In [ ]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd
import multiprocessing

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP", "CONTINUE", "10 Meters")
    return salyr

def process_batch(batch_params):
    i, j, fc_GUA_pt_50m_nw_25m, pedestrianNWpath, uc_code, nbMeters, gdb_path, gdb_name = batch_params

    try:
        expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i, j)
        if arcpy.Exists("GUA_points_layer"):
            arcpy.Delete_management("GUA_points_layer")
        GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
        print('GUA_pt_lyr')

        # Create service area layer
        salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code, i, j), pedestrianNWpath, uc_code, nbMeters)

        fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if i == 0:
            if arcpy.Exists(fc_SA):
                arcpy.Delete_management(fc_SA)
            saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
        else:
            tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(uc_code, i, j))
            arcpy.Append_management(tmpFC, saFC, "NO_TEST")
            arcpy.Delete_management(tmpFC)

        arcpy.Delete_management(salyr)

    except Exception as e:
        print(f"Error processing batch {i}-{j}: {e}")
        raise e

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
   
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 8
end_index = 9

# Define the path and name for the geodatabase
gdb_path = step2_f
gdb_name = f"SA_output_batch1_v0108_{end_index}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            
            inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))

            if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))

                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ids_ls_GQA = unique_values(uatlLyr, "FID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)

                    # Batch processing to prevent memory issues
                    batch_size = 5000
                    nbrec = int(nbrec)
                    batch_params_list = []

                    for i in range(0, nbrec, batch_size):
                        j = min(i + batch_size, nbrec)
                        batch_params = (i, j, fc_GUA_pt_50m_nw_25m, pedestrianNWpath, uc_code, nbMeters, gdb_path, gdb_name)
                        batch_params_list.append(batch_params)

                    # Use multiprocessing Pool to process the batches in parallel
                    with multiprocessing.Pool(processes=60) as pool:
                        pool.map(process_batch, batch_params_list)

                    arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)
                finally:
                    arcpy.CheckInExtension("Network")
        counter += 1



In [ ]:
import multiprocessing

In [ ]:
import sys, arcpy, time, traceback, os.path
import pandas as pd
import multiprocessing

# Other imports and existing code...

def process_batch(batch_params):
    i, j, fc_GUA_pt_50m_nw_25m, pedestrianNWpath, uc_code, nbMeters, gdb_path, gdb_name = batch_params

    try:
        expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i, j)
        if arcpy.Exists("GUA_points_layer"):
            arcpy.Delete_management("GUA_points_layer")
        GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
        print('GUA_pt_lyr')

        # Create service area layer
        salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code, i, j), pedestrianNWpath, uc_code, nbMeters)

        fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if i == 0:
            if arcpy.Exists(fc_SA):
                arcpy.Delete_management(fc_SA)
            saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
        else:
            tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(uc_code, i, j))
            arcpy.Append_management(tmpFC, saFC, "NO_TEST")
            arcpy.Delete_management(tmpFC)

        arcpy.Delete_management(salyr)

    except Exception as e:
        print(f"Error processing batch {i}-{j}: {e}")
        raise e

# Main processing loop
if __name__ == "__main__":
    # Other initial setup code...

    batch_size = 5000
    nbrec = int(nbrec)
    batch_params_list = []

    for i in range(0, nbrec, batch_size):
        j = min(i + batch_size, nbrec)
        batch_params = (i, j, fc_GUA_pt_50m_nw_25m, pedestrianNWpath, uc_code, nbMeters, gdb_path, gdb_name)
        batch_params_list.append(batch_params)

    # Use multiprocessing Pool to process the batches in parallel
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        pool.map(process_batch, batch_params_list)

    print("All batches processed.")


In [1]:
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.MakeServiceAreaLayer_na(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.AddLocations_na(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.Solve_na(salyr, "SKIP")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = "SA_output_batch1_v0108.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"

# Initialize a counter
counter = 1
start_index =6
end_index = 7

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if start_index <= counter < end_index:
            # Perform your operations with row

            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())

        
        # Break the loop after reaching the end_index
        if counter >= end_index:
            break
        counter += 1



Network Analyst extension has been checked out.
6
GEOSTAT21_029
Göteborg


In [7]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.MakeServiceAreaLayer_na(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.AddLocations_na(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.Solve_na(salyr, "SKIP")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 6
end_index = 7

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_output_batch1_v0108_{start_index}_{end_index}_polygon.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            
            ###inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            inGQA = os.path.join(indata_f, 'Processing', 'GoteborgTest.gdb', 'Polygon')
            fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))

            ###if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO":
            if arcpy.Exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))

                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ###ids_ls_GQA = unique_values(uatlLyr, "FID")
                    ids_ls_GQA = unique_values(uatlLyr, "OBJECTID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    ###fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    fields_ls = ["OBJECTID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                       
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m,  ["FID", "SHAPE@", "Area"]) as cursor:
                        ###with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
           
                    #Loop through to prevent memory errors
                    # Batch processing to prevent memory issues
                    batch_size = 5000 
                    print(f'batch size {batch_size}')

                    # Function to create service area layer
                    for i in range(0, int(nbrec), batch_size):
                        j = min(i + batch_size, int(nbrec))
                        print ("this is i {} and this is j {}".format(i,j))
                        expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                        if arcpy.Exists("GUA_points_layer"):
                            arcpy.Delete_management("GUA_points_layer")
                        GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
                        print('GUA_pt_lyr')

                        # Create service area layer
                        salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j), pedestrianNWpath, uc_code, nbMeters)

                        if i == 0:
                            fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
                            if arcpy.Exists(fc_SA):
                                arcpy.Delete_management(fc_SA)
                            #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                            saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                            
                        else:
                            tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(uc_code,i,j))
                            arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                            arcpy.Delete_management(tmpFC)

                        arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)
                finally:
                    arcpy.CheckInExtension("Network")
        counter += 1



Network Analyst extension has been checked out.
6
GEOSTAT21_029
Göteborg
start: 12:49:23
Extent set to match uc: 4431000 3837000 4448000 3855000 NaN NaN NaN NaN
nb record(s) in Polygon subset selection [code_2018] IN (14100,30000,31000) : 1
Create points from GUA borders used to create SA
GeneratePointsAlongLines Distance 50m from selected GUA in Polygon to GEOSTAT21_029_GUA_pt_50m
nb generated points : 11
SelectLayerByLocation points in GEOSTAT21_029_GUA_pt_50m WITHIN_A_DISTANCE of 25 meters from nw to GEOSTAT21_029_GUA_pt_50m_nw_25m
Nb of GQA near network 1
Total nb of GQA 1
Centroids and FIDs computed and appended successfully.
nb points in GEOSTAT21_029_GUA_pt_50m_nw_25m : 10
batch size 5000
this is i 0 and this is j 10
GUA_pt_lyr
Creating network layer for batch GEOSTAT21_029
Solving network analysis


: 

In [6]:
inGQA

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\Processing\\GoteborgTest.gdb\\Polygon'

In [5]:
fc_SA

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\OutputData\\batch1_allprocessed\\step2\\SA_output_batch1_v0108_6_7_polygon.gdb\\GEOSTAT21_029_SA'

In [ ]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve_(salyr, "SKIP")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 8
end_index = 9

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_output_batch1_v0108_{end_index}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            
            inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            fc_SA = os.path.join(step2_f, "{}_SA.shp".format(uc_code))

            if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))

                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ids_ls_GQA = unique_values(uatlLyr, "FID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
           
                    #Loop through to prevent memory errors
                    # Batch processing to prevent memory issues
                    batch_size = 15000 
                    print(f'batch size {batch_size}')

                    # Function to create service area layer
                    for i in range(0, int(nbrec), batch_size):
                        j = min(i + batch_size, int(nbrec))
                        print ("this is i {} and this is j {}".format(i,j))
                        expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                        if arcpy.Exists("GUA_points_layer"):
                            arcpy.Delete_management("GUA_points_layer")
                        GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
                        print('GUA_pt_lyr')

                        # Create service area layer
                        salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j), pedestrianNWpath, uc_code, nbMeters)

                        if i == 0:
                            if arcpy.Exists(fc_SA):
                                arcpy.Delete_management(fc_SA)
                            #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                            saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                            
                        else:
                            tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(uc_code,i,j))
                            arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                            arcpy.Delete_management(tmpFC)

                        arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)
                finally:
                    arcpy.CheckInExtension("Network")
        counter += 1



In [ ]:
import pandas as pd

In [ ]:
from multiprocessing import Pool

In [ ]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 2
end_index = 4

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_output_batch1_v0108_{end_index}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            
            inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))

            if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))

                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ids_ls_GQA = unique_values(uatlLyr, "FID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
           
                    #Loop through to prevent memory errors
                    # Batch processing to prevent memory issues
                    batch_size = 5000 
                    print(f'batch size {batch_size}')

                    # Function to create service area layer
                    for i in range(0, int(nbrec), batch_size):
                        j = min(i + batch_size, int(nbrec))
                        print ("this is i {} and this is j {}".format(i,j))
                        expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                        if arcpy.Exists("GUA_points_layer"):
                            arcpy.Delete_management("GUA_points_layer")
                        GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code), expr)
                        print('GUA_pt_lyr')

                        # Create service area layer
                        with Pool(processes=12) as pool:  # Adjust number of processes based on CPU cores
                            salyr = pool.map(create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j),
                                   pedestrianNWpath, uc_code, nbMeters), "Facilities")

                        if i == 0:
                            fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
                            if arcpy.Exists(fc_SA):
                                arcpy.Delete_management(fc_SA)
                            #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                            saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                            
                        else:
                            tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(uc_code,i,j))
                            arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                            arcpy.Delete_management(tmpFC)

                        arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)
                finally:
                    arcpy.CheckInExtension("Network")
        counter += 1



In [ ]:
with Pool(processes=12) as pool:  # Adjust number of processes based on CPU cores
    salyr = pool.map(create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j),
                                   pedestrianNWpath, uc_code, nbMeters))

    

In [ ]:
def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.MakeServiceAreaLayer_na(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.AddLocations_na(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    arcpy.Solve_na(salyr, "SKIP")
    print("Network analysis solved")
    return salyr

In [ ]:
---------------------------------------------------------------------------
CustomError                               Traceback (most recent call last)
In  [1]:
Line 106:   raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

CustomError: 'Featureclass P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\Network data Regio\\NW_CH-DE.gdb\\CH-DE\\Network_ND NOT FOUND'

In [ ]:
salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j),
                                   pedestrianNWpath, uc_code, nbMeters)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    arcpy.na.Solve(salyr, "SKIP")
    print("Network analysis solved")
    return salyr

In [ ]:
import arcpy
from multiprocessing import Pool

def create_service_area(facility):
    # Set up and run the service area analysis for a given facility
    arcpy.na.MakeServiceAreaAnalysisLayer(network_dataset, "ServiceArea", ...)
    arcpy.na.AddLocations("ServiceArea", "Facilities", facility, ...)
    arcpy.na.Solve("ServiceArea")
    # Save the output

if __name__ == "__main__":
    facilities = ["Facility1.shp", "Facility2.shp", "Facility3.shp"]  # List of facilities
    with Pool(processes=4) as pool:  # Adjust number of processes based on CPU cores
        pool.map(create_service_area, facilities)


In [ ]:
uc_code = 'GEOSTAT21_018'
fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)

In [ ]:
fc_GUA_pt_50m_nw_25m
# Create the expression with a conditional check
field_to_check = 'FID'
alternative_field = 'ORIG_FID'
expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
# Execute CalculateField
arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")


In [ ]:
arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

In [ ]:
import arcpy

# Define the path to the shapefile
shapefile_path = r"C:\path\to\your\shapefile.shp"

# Initialize variables to store the combined extent
xmin, ymin, xmax, ymax = None, None, None, None

# Check if the shapefile exists
if arcpy.Exists(shapefile_path):
    # Open a search cursor to iterate through the features
    with arcpy.da.SearchCursor(shapefile_path, ["SHAPE@"]) as cursor:
        for row in cursor:
            geom = row[0]  # Geometry object
            extent = geom.extent  # Extent object of the geometry
            
            # Update the combined extent
            if xmin is None or extent.XMin < xmin:
                xmin = extent.XMin
            if ymin is None or extent.YMin < ymin:
                ymin = extent.YMin
            if xmax is None or extent.XMax > xmax:
                xmax = extent.XMax
            if ymax is None or extent.YMax > ymax:
                ymax = extent.YMax
    
    # Set the environment extent using the combined extent
    if xmin is not None and ymin is not None and xmax is not None and ymax is not None:
        arcpy.env.extent = arcpy.Extent(xmin, ymin, xmax, ymax)
        print(f"Environment extent set to: {arcpy.env.extent}")
    else:
        print("No valid features found in the shapefile to set extent.")
else:
    print(f"Shapefile does not exist at {shapefile_path}")


In [ ]:
# 1 COMMON SOURCES FOR ALL DATA
# URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB1B2.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)
uc['CNTR_CODE'].fillna('AA', inplace=True)

# Select cities for processing in this batch
uc_sel = uc.query('Batch==1.0')
uc_sel = uc_sel.sort_values(by='CNTR_CODE')

# Read table to list the cities to process using urban centre code
cities_ls = uc_sel.HDENS_CLST.tolist()

# NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# URBAN ATLAS
# Read table with HDENS Urban centres information and Agglomerations link
HDENS_AGGL_tbl = pd.read_csv(r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\UrbanCentres_Agglomerations_csv.csv')
# Join uc code field to this table
HDENS_AGGL_tbl = HDENS_AGGL_tbl.merge(uc[['POPL_2021', 'HDENS_CLST']], on='POPL_2021')
# UA data folder
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

In [ ]:
for i in GQA_ls:
    uc_code = os.path.basename(GQA_ls[0])[:-13]
    print(uc_code)
    codes.query(f"HDENS_CLST=='{uc_code}'")

In [ ]:
GEOSTAT21_002_finalGQA.shp

In [ ]:
outCentroids = os.path.join(step2_f, f'{uc_code}_pts_GQA.shp')

In [ ]:
arcpy.CheckInExtension("Network")
if arcpy.CheckExtension("Network") == "Available":
    arcpy.CheckOutExtension("Network")
else:
    raise CustomError("The Network Analyst extension is not available.")

#Environment settings
arcpy.env.overwriteOutput = 'true'
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')
sql_query = "Batch = 1"
with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        if os.path.exists(inGQA):
            print(inGQA)
            inGQA = inGQA
            ctr_cd = ctr_cd

            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))

            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')
                print ("Points generated - hurray!!")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " +len(UATL_Ids_GUAs_nw_25m))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +len(ids_ls_GQA))

                # GQA without access to network
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                

                

                desc = arcpy.Describe(uatlLyr)
                SR = desc.spatialReference
                flds = arcpy.ListFields(uatlLyr)
                ###UATL_FieldName_List = [fld.name for fld in flds if fld.name != desc.OIDFieldName and fld.type != 'Geometry' and fld.name != desc.areaFieldName and fld.name != desc.lengthFieldName]
                UATL_FieldName_List = [fld.name for fld in flds if fld.type != 'Geometry']
                # rename FID to OBJECTID
                UATL_FieldName_List = ['OBJECTID' if field == 'FID' else field for field in UATL_FieldName_List]


                ##fc_GUA_centroids_no_pt = outPath + "\\GUA_centroids_no_pt.shp"
                fc_GUA_centroids_no_pt = os.path.join(outGDB, '{}GUA_centroids_no_pt'.format(uc_code))    
            ##    fc_GUA_centroids_no_pt = r"in_memory\GUA_centroids_no_pt"
                if arcpy.Exists(fc_GUA_centroids_no_pt):
                    arcpy.Delete_management(fc_GUA_centroids_no_pt)
                print ("CreateFeatureclass " + os.path.basename(fc_GUA_centroids_no_pt))
                ##arcpy.CreateFeatureclass_management("in_memory", "GUA_centroids_no_pt", "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                arcpy.CreateFeatureclass_management(outGDB, "{}GUA_centroids_no_pt".format(uc_code), "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                
                idxfld = 0
                for fldnam in UATL_FieldName_List:
                    for fldidx in range(0,len(flds)):
                        if fldnam == flds[fldidx].name:
                            typeF = flds[fldidx].type
                            if typeF == 'String':
                                length_fld = flds[fldidx].length
                            break
                    if typeF in ['Integer']:
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='LONG')
                    elif typeF == 'SmallInteger':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='SHORT')
                    elif typeF == 'String':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='TEXT',field_length=length_fld)
                    elif typeF == 'Double':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DOUBLE')
                    elif typeF == 'Single':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='FLOAT')
                    elif typeF == 'Date':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DATE')
                    idxfld += 1

                fields = []
                fields.append("SHAPE@")
                for fldnam in UATL_FieldName_List:
                    fields.append(fldnam)
                print ("Insert centroids where no point in " + os.path.basename('{}fc_GUA_pt_50m_nw_25m').format(uc_code))
                cursor = arcpy.da.InsertCursor(fc_GUA_centroids_no_pt, fields)  
                for row in arcpy.da.SearchCursor(uatlLyr, fields):
                    if row[1] not in UATL_Ids_GUAs_nw_25m:
                        list_row = list(row)
                        list_row[0] = row[0].centroid
                        cursor.insertRow(convert(list_row))  
                del row,cursor

                print("centroids inserted")

                ##fc_GUA_mem_pt_all = "in_memory\GUA_pt_ALL"
                fc_GUA_mem_pt_all = outGDB +"\\{}_GUA_pt_ALL".format(uc_code)
                ###fc_GUA_mem_pt_all = outPath + "\\\GUA_pt_ALL.shp"

                if arcpy.Exists(fc_GUA_mem_pt_all):
                    arcpy.Delete_management(fc_GUA_mem_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " in "  + fc_GUA_mem_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_pt_50m_nw_25m, fc_GUA_mem_pt_all) 
                print ("append " + os.path.basename(fc_GUA_centroids_no_pt) + " in " + os.path.basename(fc_GUA_mem_pt_all))
                arcpy.Append_management(fc_GUA_centroids_no_pt, fc_GUA_mem_pt_all, "NO_TEST")

                cnt = arcpy.GetCount_management(fc_GUA_mem_pt_all)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_mem_pt_all) + " : " + nbrec)

                
                #Loop through to prevent memory errors
                # Batch processing to prevent memory issues
                batch_size = 1000 
                print('batch size 1000')

                # Function to create service area layer
                for i in range(0, int(nbrec), batch_size):
                    j = min(i + batch_size, int(nbrec))
                    print ("this is i {} and this is j {}".format(i,j))
                    expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                    if arcpy.Exists("GUA_points_layer"):
                        arcpy.Delete_management("GUA_points_layer")
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_mem_pt_all, "{}GUA_points_layer".format(uc_code), expr)
                    print('GUA_pt_lyr')

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j), pedestrianNWpath, uc_code, nbMeters)

                    if i == 0:
                        fc_SA = os.path.join(gdb_path, gdb_name, "{}_sa".format(uc_code))
                        if arcpy.Exists(fc_SA):
                            arcpy.Delete_management(fc_SA)
                        #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                        saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                        
                    else:
                        tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(ctr_cd,i,j))
                        arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                        arcpy.Delete_management(tmpFC)

                    arcpy.Delete_management(salyr) 

                fc_GUA_pt_all = os.path.join(gdb_path, gdb_name, "GUA_{}_pt_ALL".format(uc_code))
                if arcpy.Exists(fc_GUA_pt_all):
                    arcpy.Delete_management(fc_GUA_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_mem_pt_all) + " in "  + fc_GUA_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_mem_pt_all, fc_GUA_pt_all)
                

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)
            finally:
                arcpy.CheckInExtension("Network")




### afternoon

In [ ]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP", "CONTINUE")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 1
end_index = 20

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_output_batch1_v0108_{end_index}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            print(ctr_cd)
            inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
            ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
            if ctr_cd.startswith("CH-"):
                ctr_cd = "CH"

            if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))
                
                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ids_ls_GQA = unique_values(uatlLyr, "FID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
           
                    if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                        arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                    print("{}GUA_points_layer".format(uc_code))

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                    arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)

        counter += 1



In [ ]:
arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)

In [ ]:
fc_SA

In [ ]:
fc_SA = os.path.join(step2_f, "{}_SA.shp".format(uc_code))
arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)

In [ ]:
fc_SA

In [ ]:
fc_SA

In [ ]:
step2_f

In [ ]:
if ctr_cd =="CH-*":
    ctr_cd =="CH"

In [ ]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP", "CONTINUE")
    return salyr

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400
    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for Batch N
sql_query = "Batch = 1"
# Initialize a counter
counter = 1
start_index = 1
end_index = 20

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_output_batch1_v0108_{end_index}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        if counter >= end_index: # if counter out of range, skip
            break
        if start_index <= counter < end_index:
            geom = uc_row[0]  # Geometry object
            ctr_cd = uc_row[2]
            uc_code = uc_row[1]
            print (counter)
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            print(ctr_cd)
            inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
            fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
            ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
            if ctr_cd.startswith("CH-"):
                ctr_cd = "CH"

            if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
                print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
                pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
                if not arcpy.Exists(pedestrianNWpath):
                    raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

                arcpy.env.extent = geom.extent # Extent object of the geometry
                print("Extent set to match uc: {}".format(arcpy.env.extent))
                
                try:          

                    if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                        arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                    uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                    ## Count number of GUA Blocks
                    cnt = arcpy.GetCount_management(uatlLyr)
                    print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                    ## Create and Select points from GUA borders used to create SA
                    print ("Create points from GUA borders used to create SA")

                    #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                    fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                    if arcpy.Exists(fc_GUA_pt_50m):
                        arcpy.Delete_management(fc_GUA_pt_50m)
                    print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                    arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                    nbrec = cnt.getOutput(0)
                    print ("nb generated points : " + nbrec)

                    if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                        arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                    arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                    fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                    if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                        arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                    nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                    print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                    Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                    # save selection in a diferent feature class
                    arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                    # Adding FID field to the point feature class if it doesn't already exist
                    if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                        arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                    UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                    print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                    ids_ls_GQA = unique_values(uatlLyr, "FID")
                    print("Total nb of GQA " +str(len(ids_ls_GQA)))

                    # GQA without access to network (less than 25m)
                    ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                    
                    # List to hold centroid points and their corresponding FIDs
                    centroid_points = []
                    fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                    # Create a search cursor to iterate through the polygon shapefile GQA
                    with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                        for row in cursor:
                            if row[0] in ids_noaccess:
                                # Get the FID and the geometry of the polygon
                                fid = row[0]
                                polygon = row[1]
                                area = row[2]
                                # Compute the centroid of the polygon
                                centroid = polygon.centroid
                                # Append the FID and centroid to the list
                                centroid_points.append((fid, centroid, area))

                    # Start an edit session for the geodatabase
                    with arcpy.da.Editor(outGDB) as edit:
                        # Create an insert cursor for the point feature class
                        with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                            for fid, point, area in centroid_points:
                                # Insert the centroid point and the FID
                                cursor.insertRow([fid, point, area])
                    print("Centroids and FIDs computed and appended successfully.")

                    # Add FID value to all records
                    field_to_check = 'FID'
                    alternative_field = 'ORIG_FID'
                    #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                    expression ="!FID! if !FID! is not None else !ORIG_FID!"
                    # Execute CalculateField
                    arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                    cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                    nbrec = cnt.getOutput(0)
                    print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
           
                    if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                        arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                    print("{}GUA_points_layer".format(uc_code))

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                    arcpy.Delete_management(salyr) 

                    print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                    
                except CustomError as ce:
                    arcpy.AddError(ce.value)
                    print (ce.value)
                except arcpy.ExecuteError:
                    msgs = arcpy.GetMessages(2)
                    arcpy.AddError(msgs)
                    print (msgs)
                except:
                    tb = sys.exc_info()[2]
                    tbinfo = traceback.format_tb(tb)[0]
                    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                    arcpy.AddError(pymsg)
                    print (msgs)
                    arcpy.AddError(msgs)
                    print (pymsg)

        counter += 1



# testing with 'pedestrian'

In [1]:
import sys, arcpy, time, traceback, os.path
import pandas as pd

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)

# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)
codes.query("HDENS_CLST=='GEOSTAT21_595'")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,FREQUENCY,HDENS_CLST,HDENS_NAME,HDENS_2011,agglomerationId_identifier,agglomerationName_nameEng,UA2018
303,1,GEOSTAT21_595,Bordeaux,GEOSTAT11_595,AG_FR_00_5,Bordeaux,FR007L2_BORDEAUX_UA2018_v013


In [3]:
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3.shp')
#sql_query = "Batch = 1"
sql_query = "HDENS_CLST = 'GEOSTAT21_595'"

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())

GEOSTAT21_595
Bordeaux


In [4]:
fc_GUA_pt_50m_nw_25m = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Test_SA.gdb\fc_GUA_pt_50m_nw_25m'

GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))

In [ ]:
def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):

In [6]:
network_path = pedestrianNWpath
nb_meters = 400
salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(uc_code), "Meters", "TRAVEL_FROM", 
                                        nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", 
                                        "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")

In [14]:
travel_modes = arcpy.nax.GetTravelModes(network_path)
# Construct a new TravelMode object from the existing Driving Time travel mode
new_travel_mode = arcpy.nax.TravelMode(travel_modes["Driving Time"])
# Update the useHierarchy property to turn hierarchy off, and update the name
new_travel_mode.name = "Test without Hierarchy"
new_travel_mode.type = "Walk"
new_travel_mode.useHierarchy = "NO_HIERARCHY"
# Use the new travel mode object when constructing an OD cost matrix analysis
od_object = arcpy.nax.OriginDestinationCostMatrix(network_path)
od_object.travelMode = new_travel_mode

KeyError: 'Driving Time'

In [15]:
if 'Driving Time' in travel_modes:
    new_travel_mode = arcpy.nax.TravelMode(travel_modes['Driving Time'])
else:
    # Create a new travel mode or use a different available one
    new_travel_mode = arcpy.nax.TravelMode(travel_modes['Default'])  # Replace 'Default' with an existing mode


KeyError: 'Default'

In [19]:
network_dataset = pedestrianNWpath
salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
    network_data_source= network_dataset,
    layer_name="Service Area VCS 2",
    travel_mode="Walk",
    travel_direction="FROM_FACILITIES",
    cutoffs=[400],
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    output_type="POLYGONS",
    polygon_detail="HIGH",
    geometry_at_overlaps="OVERLAP",
    geometry_at_cutoffs="DISKS",
    polygon_trim_distance="100 Meters",
    exclude_sources_from_polygon_generation=None,
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

In [20]:
arcpy.na.AddLocations(
    in_network_analysis_layer=salyr,
    sub_layer="Facilities",
    in_table=r"P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Test_SA.gdb\fc_GUA_pt_50m_nw_25m",
    field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
    search_tolerance="5000 Meters",
    sort_field=None,
    search_criteria="nw SHAPE;Network_ND_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="NO_SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)


<Result 'Service Area VCS 2'>

In [24]:
arcpy.na.Solve(
    in_network_analysis_layer=salyr,
    ignore_invalids="SKIP",
    terminate_on_solve_error="CONTINUE",
    simplification_tolerance=None,
    overrides=""
)

<Result 'Service Area VCS 2'>

In [25]:
salyr.saveACopy(r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\salyr101024.shp')

AttributeError: 'Result' object has no attribute 'saveACopy'

# Final version updated SA, Addlocs and Solve

In [ ]:
def create_service_area_layer(input_layer, network_path, group_id):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaLayer(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.na.AddLocations(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.na.Solve(salyr, "SKIP", "CONTINUE")
    return salyr

In [ ]:
# Variables
network_dataset = pedestrianNWpath
in_table = input_layer ## locations (access points)
## in_table=r"P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Test_SA.gdb\fc_GUA_pt_50m_nw_25m"

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 



In [2]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'step1_GQA')
step2_f = os.path.join(indata_f, 'OutputData', 'step2_GQA_Final')
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for ctry
ctry = 'CH'
sql_query = "Batch >0 "

# Define the path and name for the geodatabase
gdb_path = os.path.join(indata_f, 'Processing')

gdb_name = f"SA_output_2510_{ctry}2.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
        ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if ctr_cd.startswith("CH-"):
            ctr_cd = "CH"

        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
        if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd==ctry:
            print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))
            
            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                # save selection in a diferent feature class
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                # Adding FID field to the point feature class if it doesn't already exist
                if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                    arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +str(len(ids_ls_GQA)))

                # GQA without access to network (less than 25m)
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                
                # List to hold centroid points and their corresponding FIDs
                centroid_points = []
                fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                # Create a search cursor to iterate through the polygon shapefile GQA
                with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                    for row in cursor:
                        if row[0] in ids_noaccess:
                            # Get the FID and the geometry of the polygon
                            fid = row[0]
                            polygon = row[1]
                            area = row[2]
                            # Compute the centroid of the polygon
                            centroid = polygon.centroid
                            # Append the FID and centroid to the list
                            centroid_points.append((fid, centroid, area))

                # Start an edit session for the geodatabase
                with arcpy.da.Editor(outGDB) as edit:
                    # Create an insert cursor for the point feature class
                    with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                        for fid, point, area in centroid_points:
                            # Insert the centroid point and the FID
                            cursor.insertRow([fid, point, area])
                print("Centroids and FIDs computed and appended successfully.")

                # Add FID value to all records
                field_to_check = 'FID'
                alternative_field = 'ORIG_FID'
                #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                expression ="!FID! if !FID! is not None else !ORIG_FID!"
                # Execute CalculateField
                arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
        
                if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                    arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                print("{}GUA_points_layer".format(uc_code))
                

                # Create service area layer
                ###create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                group_id = uc_code
                network_dataset = pedestrianNWpath
                locations = GUA_pt_lyr
                create_service_area_layer(group_id, network_dataset, locations, fc_SA)

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)





Network Analyst extension has been checked out.
GEOSTAT21_018
Stockholm
GEOSTAT21_021
Tartu
GEOSTAT21_022
Örebro
GEOSTAT21_023
Stavanger
GEOSTAT21_024
Norrköping
GEOSTAT21_025
Linköping
GEOSTAT21_914
Jönköping / Huskvarna
GEOSTAT21_029
Göteborg
GEOSTAT21_026
Borås
GEOSTAT21_028
Rīga
GEOSTAT21_031
Aalborg
GEOSTAT21_047
Århus
GEOSTAT21_048
Helsingborg
GEOSTAT21_903
København
GEOSTAT21_049
Lund
GEOSTAT21_051
Kaunas
GEOSTAT21_054
Malmö
GEOSTAT21_060
Odense
GEOSTAT21_065
Gdynia
GEOSTAT21_068
Gdańsk
GEOSTAT21_904
Dublin
GEOSTAT21_070
Elbląg
GEOSTAT21_086
Kiel
GEOSTAT21_090
Olsztyn
GEOSTAT21_093
Lütten Klein
GEOSTAT21_094
Rostock
GEOSTAT21_095
Limerick
GEOSTAT21_108
Lübeck
GEOSTAT21_109
Białystok
GEOSTAT21_119
Hamburg
GEOSTAT21_118
Bremerhaven
GEOSTAT21_120
Szczecin
GEOSTAT21_919
Neu Wulmstorf / Neugraben-Fischbek / Hausbruch
GEOSTAT21_121
Harburg
GEOSTAT21_122
Kasztelanka
GEOSTAT21_128
Bydgoszcz
GEOSTAT21_132
Cork
GEOSTAT21_131
Toruń
GEOSTAT21_137
Groningen
GEOSTAT21_141
Oldenburg (Oldenburg

In [ ]:
#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="100 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 



In [ ]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step3_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

### final final final

In [26]:
#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 

In [ ]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
step2_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
GQA_f = os.path.join(indata_f, 'OutputData', 'GQA')

step3_f = os.path.join(indata_f, 'OutputData', 'oct_run', 'step2_SA_final')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")

In [ ]:
# Run process for ctry
sql_query = "Batch >0"
nb_meters = 400

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_1610_vMS.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for i, uc_row in enumerate(uc_cursor):
        if i < 500:
            continue  # Skip the first 9 rows
        if i >= 700:
            break  # Stop after row 25

        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]

        inGQA = os.path.join(GQA_f, '{}_finalGQA.shp'.format(uc_code))
        fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
        ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if ctr_cd.startswith("CH-"):
            ctr_cd = "CH"

        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd==ctry:
        if os.path.exists(inGQA) and not arcpy.Exists(fc_SA):
            print(uc_code)
            print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
            print(ctr_cd)
            print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))
            
            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                # save selection in a diferent feature class
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                # Adding FID field to the point feature class if it doesn't already exist
                if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                    arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +str(len(ids_ls_GQA)))

                # GQA without access to network (less than 25m)
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                
                # List to hold centroid points and their corresponding FIDs
                centroid_points = []
                fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
                # Create a search cursor to iterate through the polygon shapefile GQA
                with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                    for row in cursor:
                        if row[0] in ids_noaccess:
                            # Get the FID and the geometry of the polygon
                            fid = row[0]
                            polygon = row[1]
                            area = row[2]
                            # Compute the centroid of the polygon
                            centroid = polygon.centroid
                            # Append the FID and centroid to the list
                            centroid_points.append((fid, centroid, area))

                # Start an edit session for the geodatabase
                with arcpy.da.Editor(outGDB) as edit:
                    # Create an insert cursor for the point feature class
                    with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                        for fid, point, area in centroid_points:
                            # Insert the centroid point and the FID
                            cursor.insertRow([fid, point, area])
                print("Centroids and FIDs computed and appended successfully.")

                # Add FID value to all records
                field_to_check = 'FID'
                alternative_field = 'ORIG_FID'
                #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"                
                expression ="!FID! if !FID! is not None else !ORIG_FID!"

                
                # Execute CalculateField
                arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
        
                if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                    arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                print("{}GUA_points_layer".format(uc_code))
                

                # Create service area layer
                ###create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                group_id = uc_code
                network_dataset = pedestrianNWpath
                locations = GUA_pt_lyr
                create_service_area_layer(group_id, network_dataset, locations, fc_SA)

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)

In [ ]:
import arcpy
from multiprocessing import Pool

def run_geoprocessing(task):
    # Example function to run a geoprocessing tool
    arcpy.Buffer_analysis(task['input'], task['output'], task['distance'])

if __name__ == '__main__':
    tasks = [
        {'input': 'feature1.shp', 'output': 'buffer1.shp', 'distance': '100 meters'},
        {'input': 'feature2.shp', 'output': 'buffer2.shp', 'distance': '200 meters'},
    ]
    with Pool(processes=4) as pool:
        pool.map(run_geoprocessing, tasks)

In [ ]:
import arcpy
from multiprocessing import Pool

def run_geoprocessing(task):
    # Example function to run a geoprocessing tool
    arcpy.Buffer_analysis(task['input'], task['output'], task['distance'])

if __name__ == '__main__':
    tasks = [
        {'input': 'feature1.shp', 'output': 'buffer1.shp', 'distance': '100 meters'},
        {'input': 'feature2.shp', 'output': 'buffer2.shp', 'distance': '200 meters'},
    ]
    with Pool(processes=4) as pool:
        pool.map(run_geoprocessing, tasks)

In [ ]:
def createSA(uc_row):
    geom = uc_row[0]  # Geometry object
    ctr_cd = uc_row[2]
    uc_code = uc_row[1]

    inGQA = os.path.join(GQA_f, '{}_finalGQA.shp'.format(uc_code))
    fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
    ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
    if ctr_cd.startswith("CH-"):
        ctr_cd = "CH"

    ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
    ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd==ctry:
    if os.path.exists(inGQA) and not arcpy.Exists(fc_SA):
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        print(ctr_cd)
        print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
        pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
        if not arcpy.Exists(pedestrianNWpath):
            raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

        arcpy.env.extent = geom.extent # Extent object of the geometry
        print("Extent set to match uc: {}".format(arcpy.env.extent))
        
        try:          

            if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                arcpy.Delete_management("{}_UATLlayer".format(uc_code))
            uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

            ## Count number of GUA Blocks
            cnt = arcpy.GetCount_management(uatlLyr)
            print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

            ## Create and Select points from GUA borders used to create SA
            print ("Create points from GUA borders used to create SA")

            #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
            fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

            if arcpy.Exists(fc_GUA_pt_50m):
                arcpy.Delete_management(fc_GUA_pt_50m)
            print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
            arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

            cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
            nbrec = cnt.getOutput(0)
            print ("nb generated points : " + nbrec)

            if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
            arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

            fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
            if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
            nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
            print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
            Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
            # save selection in a diferent feature class
            arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
            # Adding FID field to the point feature class if it doesn't already exist
            if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

            UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
            print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
            ids_ls_GQA = unique_values(uatlLyr, "FID")
            print("Total nb of GQA " +str(len(ids_ls_GQA)))

            # GQA without access to network (less than 25m)
            ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
            
            # List to hold centroid points and their corresponding FIDs
            centroid_points = []
            fields_ls = ["FID", "SHAPE@", "Area"] # List of fields to include
            # Create a search cursor to iterate through the polygon shapefile GQA
            with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                for row in cursor:
                    if row[0] in ids_noaccess:
                        # Get the FID and the geometry of the polygon
                        fid = row[0]
                        polygon = row[1]
                        area = row[2]
                        # Compute the centroid of the polygon
                        centroid = polygon.centroid
                        # Append the FID and centroid to the list
                        centroid_points.append((fid, centroid, area))

            # Start an edit session for the geodatabase
            with arcpy.da.Editor(outGDB) as edit:
                # Create an insert cursor for the point feature class
                with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                    for fid, point, area in centroid_points:
                        # Insert the centroid point and the FID
                        cursor.insertRow([fid, point, area])
            print("Centroids and FIDs computed and appended successfully.")

            # Add FID value to all records
            field_to_check = 'FID'
            alternative_field = 'ORIG_FID'
            #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"                
            expression ="!FID! if !FID! is not None else !ORIG_FID!"

            
            # Execute CalculateField
            arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

            cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
            nbrec = cnt.getOutput(0)
            print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
    
            if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
            GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
            print("{}GUA_points_layer".format(uc_code))
            

            # Create service area layer
            ###create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
            group_id = uc_code
            network_dataset = pedestrianNWpath
            locations = GUA_pt_lyr
            create_service_area_layer(group_id, network_dataset, locations, fc_SA)

            print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
            
        except CustomError as ce:
            arcpy.AddError(ce.value)
            print (ce.value)
        except arcpy.ExecuteError:
            msgs = arcpy.GetMessages(2)
            arcpy.AddError(msgs)
            print (msgs)
        except:
            tb = sys.exc_info()[2]
            tbinfo = traceback.format_tb(tb)[0]
            pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
            msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
            arcpy.AddError(pymsg)
            print (msgs)
            arcpy.AddError(msgs)
            print (pymsg)

In [ ]:
# Run process for ctry
sql_query = "Batch >0 "
nb_meters = 400

# Define the path and name for the geodatabase
gdb_path = outPath
gdb_name = f"SA_1610.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for i, uc_row in enumerate(uc_cursor):
        if i >= 100:
            break
        createSA(uc_row)

